In [80]:
import sqlite3
conn = sqlite3.connect('/Users/oj/Documents/twitter_db/sentiment.db')
cur = conn.cursor()

In [81]:
def procesed_text(cur, limit):
    rows = cur.execute("""
        select processedUrl_UserName_HashTag from second_run limit {limit};
        """.format(limit=limit))
    for row in rows:
        yield row[0]

def sentiment_value(cur, limit):
    rows = cur.execute("""
        select score from second_run limit {limit};
        """.format(limit=limit))
    for row in rows:
        yield row[0]

In [82]:
from sklearn.pipeline import Pipeline
text_clf = Pipeline([('vect', CountVectorizer()),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultinomialNB()),
])

In [101]:
row_count = 25000
data = list(procesed_text(cur, row_count))
target = list(sentiment_value(cur, row_count))
text_clf = text_clf.fit(data, target)

In [102]:
print(len([x for x in target if x == '0']))

23080


In [104]:
docs_new = ['God is love', 'OpenGL on the GPU is fast', 'i hate this', 'this is good', 'this was terrible']
predicted = text_clf.predict(docs_new)

for tweet, sentiment in zip(docs_new, predicted):
    print('%r => %s' % (tweet, sentiment))

'God is love' => 1
'OpenGL on the GPU is fast' => 1
'i hate this' => 0
'this is good' => 1
'this was terrible' => 0


In [105]:
import pickle
s = pickle.dumps(text_clf)
clf2 = pickle.loads(s)
clf2.predict(['best day ever'])

array(['1'], 
      dtype='<U1')

In [107]:
import os
os.getcwd()

'/Users/oj/Documents/git/ga_data_science/app/data'

In [108]:
from sklearn.externals import joblib
joblib.dump(text_clf, 'filename.pkl') 

['filename.pkl',
 'filename.pkl_01.npy',
 'filename.pkl_02.npy',
 'filename.pkl_03.npy',
 'filename.pkl_04.npy',
 'filename.pkl_05.npy',
 'filename.pkl_06.npy',
 'filename.pkl_07.npy']

In [109]:
clf3 = joblib.load('filename.pkl') 

In [110]:
clf3.predict(['worst day ever'])

array(['0'], 
      dtype='<U1')